
# Linear Algebra Foundations for Quantum Computing — Chapter 1

This notebook introduces the **linear algebra** you need for quantum computing.
Each concept is explained briefly and **immediately demonstrated in Python** using `numpy`, with math written in **MyST** syntax for Jupyter Book.



## 1. Motivation & Notation

Quantum states live in **complex vector spaces** and evolve by **linear (unitary) maps**.  
We need fluency with **vectors, matrices, inner products**, and the **tensor product**.

**Notations you'll see**
- Array notation (Python / math): vectors as column arrays, matrices as 2D arrays.  
- Dirac notation: {math}`|v\rangle` for a column vector ("ket"), and {math}`\langle v|` for its conjugate transpose ("bra").  
- We’ll use standard Python arrays, with the understanding that a column vector {math}`|v\rangle` is represented as shape `(n, 1)` or a 1-D array of length `n`.


In [ ]:

import numpy as np
np.set_printoptions(precision=4, suppress=True)



## 2. Vectors

A **vector** is an ordered list of numbers (often complex) arranged in a column.  
The **dimension** of a vector space is the length of the coordinate list relative to a basis.

- Standard basis of {math}`\mathbb{C}^n`: {math}`e_1, \dots, e_n` where {math}`e_i` has a 1 in position {math}`i` and 0 elsewhere.  
- The **norm** (length) of {math}`x` is {math}`\|x\|_2 = \sqrt{\sum_i |x_i|^2}`.  
- A **unit vector** has norm 1; **normalization** rescales {math}`x` to {math}`x/\|x\|` (if {math}`x \neq 0`).

> **Textbook tip:** In quantum, we typically normalize state vectors to unit length so that total probability is 1.


In [ ]:

# Create vectors (1-D) and column vectors (2-D shape n x 1)
x = np.array([3, 4], dtype=float)              # 1-D vector in R^2
x_col = x.reshape(-1, 1)                        # column form (2,1)
e1 = np.array([1, 0], dtype=float)
e2 = np.array([0, 1], dtype=float)

def l2_norm(v):
    v = np.asarray(v).reshape(-1)
    return np.sqrt(np.vdot(v, v))  # vdot conjugates the first arg (safe for complex)

print("x =", x, " shape:", x.shape)
print("x as column:\n", x_col, " shape:", x_col.shape)
print("||x||2 =", l2_norm(x))
print("Basis e1, e2 ->", e1, e2)

# Normalize x
x_unit = x / l2_norm(x)
print("Normalized x:", x_unit, "||x_unit||2 =", l2_norm(x_unit))



## 3. Complex Numbers & Conjugation

Quantum amplitudes are complex. For {math}`z = a + ib`:

- **Conjugate**: {math}`\overline{z} = a - ib`  
- **Modulus**: {math}`|z| = \sqrt{a^2 + b^2}`  
- **Euler**: {math}`re^{i\theta} = r(\cos\theta + i\sin\theta)`  

For vectors/matrices, the **conjugate transpose** (Hermitian adjoint) is denoted by {math}`(\cdot)^\dagger`.


In [ ]:

z = 3 + 4j
print("z =", z, "conjugate:", np.conj(z), "|z| =", np.abs(z))

# Complex vector
v = np.array([1+2j, 2-1j], dtype=complex)
print("v =", v)
print("v* (elementwise conj) =", v.conj())
print("v^† (as a row) =", v.conj().reshape(1,-1))  # bra



## 4. Vector Dot (Inner) Product

For {math}`u, v \in \mathbb{C}^n`, the inner product is
{math}`\langle u, v\rangle = \sum_i \overline{u_i}\, v_i`.

Properties:
- Conjugate symmetry: {math}`\langle u, v\rangle = \overline{\langle v, u\rangle}`  
- Linearity in second argument  
- {math}`\langle v, v\rangle = \|v\|^2 \ge 0`, and equals 0 iff {math}`v = 0`  
- **Orthogonality**: {math}`\langle u, v\rangle = 0`

**Python tips**
- `np.vdot(u, v)` computes {math}`\overline{u}^\top v` (conjugates the first argument).  
- `u @ v` for 1-D arrays computes the Euclidean dot without conjugation; for complex vectors, prefer `np.vdot`.


In [ ]:

u = np.array([1+1j, 0-1j], dtype=complex)
v = np.array([2-1j, 3+0j], dtype=complex)

print("u =", u)
print("v =", v)
print("<u, v> via vdot =", np.vdot(u, v))
print("<v, u> via vdot =", np.vdot(v, u), " (conjugate of previous)")
print("||u||^2 =", np.vdot(u,u).real)
print("Orthogonality check <u, [i*u]> :", np.vdot(u, 1j*u))  # equals i * ||u||^2, not orthogonal



## 5. Matrices

A **matrix** represents a linear map. Key notions:
- **Identity** {math}`I` satisfies {math}`Ix = x`.  
- **Transpose** {math}`A^\top`, **conjugate** {math}`\overline{A}`, and **conjugate transpose** (Hermitian adjoint) {math}`A^\dagger = \overline{A}^\top`.  
- **Hermitian**: {math}`A = A^\dagger`.  
- **Unitary**: {math}`U^\dagger U = I` (length-preserving).


In [ ]:

A = np.array([[1, 2], [3, 4]], dtype=float)
B = np.array([[0, -1j], [1j, 0]], dtype=complex)  # skew-Hermitian example

I = np.eye(2)
print("A =\n", A)
print("A^T =\n", A.T)

print("B =\n", B)
print("B^† =\n", B.conj().T)

# Hermitian example
H = np.array([[2, 1-1j],[1+1j, 3]], dtype=complex)
print("H Hermitian? ->", np.allclose(H, H.conj().T))

# Unitary example: 2x2 Hadamard (scaled)
H2 = (1/np.sqrt(2)) * np.array([[1, 1],[1, -1]], dtype=float)
print("H2 unitary? ->", np.allclose(H2.conj().T @ H2, np.eye(2)))



## 6. Matrix–Vector & Matrix–Matrix Products

- **Matrix–vector**: {math}`y = A x` applies a linear map to a vector.  
- **Matrix–matrix**: {math}`C = AB` composes linear maps (first {math}`B`, then {math}`A`).  
- **Associativity** holds: {math}`A(Bx) = (AB)x`; **order matters** (in general {math}`AB \ne BA`).


In [ ]:

A = np.array([[2, 0],[0, 3]], dtype=float)  # scales x and y components
x = np.array([1, -2], dtype=float)

y = A @ x
print("A =\n", A)
print("x =", x)
print("A @ x =", y)

B = np.array([[0, 1],[1, 0]], dtype=float)   # swap matrix
print("AB =\n", A @ B)
print("BA =\n", B @ A)
print("AB == BA ?", np.allclose(A@B, B@A))   # generally False



## 7. Matrix “Dot Product” (Frobenius / Hilbert–Schmidt)

The natural inner product on matrices is
{math}`\langle A, B\rangle_F = \mathrm{Tr}(A^\dagger B) = \sum_{i,j} \overline{A_{ij}}\, B_{ij}`.

This induces the **Frobenius norm** {math}`\|A\|_F = \sqrt{\langle A, A\rangle_F}`.  
It generalizes vector dot products and is ubiquitous in quantum (e.g., overlaps, distances).

> **Python tip:** `np.vdot(A, B)` flattens and conjugates the first argument, effectively computing the same sum.


In [ ]:

A = np.array([[1+1j, 2-1j],[0+2j, 3]], dtype=complex)
B = np.array([[2, 0-1j],[1+0j, 4-2j]], dtype=complex)

fro_vdot = np.vdot(A, B)                         # sum(conj(A)*B) over all entries
fro_trace = np.trace(A.conj().T @ B)             # Tr(A^† B)

print("<A,B>_F via vdot:", fro_vdot)
print("<A,B>_F via trace:", fro_trace)
print("Equal? ->", np.allclose(fro_vdot, fro_trace))

fro_norm = np.sqrt(np.vdot(A, A).real)
print("||A||_F =", fro_norm)



## 8. Orthonormal Bases & Projections

A set {math}`\{u_i\}` is **orthonormal** if {math}`\langle u_i, u_j\rangle = \delta_{ij}`.  
The **projection** onto a unit vector {math}`u` is {math}`P = u\,u^\dagger`, and {math}`Px` is the component of {math}`x` along {math}`u`.

> **Quantum link:** Projectors model ideal measurements onto subspaces.


In [ ]:

# Build an orthonormal pair in R^2 (Hadamard columns are orthonormal)
u = (1/np.sqrt(2)) * np.array([1, 1], dtype=float)
v = (1/np.sqrt(2)) * np.array([1,-1], dtype=float)

print("<u,u> =", np.vdot(u,u))
print("<v,v> =", np.vdot(v,v))
print("<u,v> =", np.vdot(u,v))

# Projection of x onto u
x = np.array([2.0, 1.0])
P = np.outer(u, u)    # u u^T because u is real; for complex use u[:,None] @ u[None,:].conj()
Px = P @ x

print("x =", x)
print("Projection matrix P = u u^T =\n", P)
print("Projected Px =", Px)



## 9. Tensor (Kronecker) Product

The **tensor product** builds a composite space:  
If {math}`x \in \mathbb{C}^{m}` and {math}`y \in \mathbb{C}^{n}`, then {math}`x \otimes y \in \mathbb{C}^{mn}`.  
In quantum, this combines subsystems (e.g., two qubits).

- For vectors: basis ordering {math}`|i\rangle \otimes |j\rangle = |ij\rangle`.  
- For matrices: {math}`(A \otimes B)(x \otimes y) = (Ax) \otimes (By)`.


In [ ]:

# Vector tensor product
x = np.array([1, 2], dtype=float)
y = np.array([3, 4], dtype=float)
x_otimes_y = np.kron(x, y)
print("x =", x)
print("y =", y)
print("x ⊗ y =", x_otimes_y, " shape:", x_otimes_y.shape)

# Matrix tensor product (useful for multi-qubit gates)
H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]], dtype=float)  # Hadamard
I = np.eye(2)
H2 = np.kron(H, I)  # Apply H to first qubit, identity to second
print("H ⊗ I =\n", H2)

# Check (A⊗B)(x⊗y) = (Ax)⊗(By)
A = np.array([[0,1],[1,0]], dtype=float)  # X gate
B = np.array([[1,0],[0,1]], dtype=float)  # I
lhs = np.kron(A,B) @ np.kron([1,0],[1,0])  # (A⊗B)(|0>⊗|0>)
rhs = np.kron(A @ np.array([1,0]), B @ np.array([1,0]))
print("LHS == RHS ?", np.allclose(lhs, rhs))



## 10. Unitarity & Norm Preservation

Unitary matrices preserve inner products and norms: {math}`\|Ux\| = \|x\|`.  
This is essential in quantum mechanics for probability conservation.


In [ ]:

def is_unitary(U, tol=1e-10):
    return np.allclose(U.conj().T @ U, np.eye(U.shape[0]), atol=tol)

U = (1/np.sqrt(2))*np.array([[1,1],[1,-1]], dtype=complex)  # Hadamard
x = np.array([1+1j, 2-1j], dtype=complex)

print("Is U unitary?", is_unitary(U))
print("||x|| =", np.sqrt(np.vdot(x,x)))
print("||Ux|| =", np.sqrt(np.vdot(U@x, U@x)))



## 11. Mini-Exercises (with solutions below)

1. **Normalize** {math}`w = (3, 4i)` and verify its {math}`\ell_2` norm is 1.  
2. For {math}`a=(1, i)` and {math}`b=(1, -i)`, compute {math}`\langle a, b\rangle`. Are they orthogonal?  
3. Build the **projection** onto {math}`u=\frac{1}{\sqrt{2}}(1,1)` and apply it to {math}`x=(2,1)`.  
4. Compute {math}`|0\rangle \otimes |1\rangle` where {math}`|0\rangle=(1,0)`, {math}`|1\rangle=(0,1)`.  
5. Check whether {math}`U=\begin{bmatrix}0&1\\1&0\end{bmatrix}` (the **X gate**) is unitary.



### Solutions


In [ ]:

# 1) Normalize w = (3, 4i)
w = np.array([3+0j, 0+4j])
w_unit = w / np.sqrt(np.vdot(w,w))
print("w_unit =", w_unit, " ||w_unit|| =", np.sqrt(np.vdot(w_unit,w_unit)))

# 2) Inner product of a and b
a = np.array([1+0j, 0+1j])
b = np.array([1+0j, 0-1j])
print("<a,b> =", np.vdot(a,b), "  (zero => orthogonal)")

# 3) Projection onto u and apply to x
u = (1/np.sqrt(2)) * np.array([1,1], dtype=float)
P = np.outer(u,u)  # real example
x = np.array([2.0,1.0])
print("Projection Px =", P @ x)

# 4) Tensor of |0> and |1>
zero = np.array([1,0], dtype=int)
one = np.array([0,1], dtype=int)
print("|0>⊗|1> =", np.kron(zero, one))

# 5) Unitarity of X
X = np.array([[0,1],[1,0]], dtype=complex)
print("X unitary?", np.allclose(X.conj().T @ X, np.eye(2)))



---

**Next steps:** With these tools, you can confidently read and write the linear algebra that underpins quantum states, measurements, and gates.
